In [1]:
%use dataframe

In [2]:
// This file contains LastFM scrobble data
// from: Saturday, January 1, 2022 12:00:00 AM
// until: Wednesday, November 29, 2023 9:17:54 PM
val df = DataFrame.read("scrobbles-zubie7a-1701292674.csv")

In [3]:
df.head(5)

DataFrame: rowsCount = 5, columnsCount = 8

In [4]:
// Boundary epoch values for time range
// 1640995200: Saturday, January 1, 2022 12:00:00 AM
// 1668165071: November 11, 2022 11:11:11 AM
// 1640995200: January 1, 2023 00:00:00 AM
// 1699701071: Saturday, November 11, 2023 11:11:11 AM
val start2022 = 1640995200
val end2022 = 1668165071
val start2023 = 1672531200
val end2023 = 1699701071
// Create "wrapped" dataframes between January 1 and November 11th,
// trying to mimic Spotify's time range (sorry December!)
val dfWrapped2022 = df.filter { start2022 < uts && uts < end2022 }
val dfWrapped2023 = df.filter { start2023 < uts && uts < end2023 }

In [5]:
// Find the 10 most listened songs in 2022 and 2023, to compare.

val dfMostListenedSongs2022 = 
    dfWrapped2022
        .groupBy("track")
        .count()
        .rename("track").into("track2022")
        .rename("count").into("countYear2022")
        .sortByDesc("countYear2022")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")


val dfMostListenedSongs2023 = 
    dfWrapped2023
        .groupBy("track")
        .count()
        .rename("track").into("track2023")
        .rename("count").into("countYear2023")
        .sortByDesc("countYear2023")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")

In [6]:
val dfRanksSongs2022 = 
    dfMostListenedSongs2022
        .select { rank and track2022 }
        .rename("rank").into { "track2023_rank2022" }
        .rename("track2022").into { "track" }

In [7]:
// With the left join, tracks from 2023 not present in 2022 will have
// the 2022 values filled in with null.
val dfMostListenedSongs2023WithPreviousYearRank =
    dfMostListenedSongs2023.leftJoin(dfRanksSongs2022) { track2023 match right.track }

dfMostListenedSongs2022
    .join(
        dfMostListenedSongs2023WithPreviousYearRank
            .add("track2023_rank2023") { rank } 
    )

... showing only top 20 of 6771 rows DataFrame: rowsCount = 6771, columnsCount = 7

In [8]:
// Find the 10 most listened albums in 2022 and 2023, to compare.

val dfMostListenedAlbums2022 = 
    dfWrapped2022
        .groupBy("album")
        .count()
        .rename("album").into("album2022")
        .rename("count").into("countYear2022")
        .sortByDesc("countYear2022")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")
        
val dfMostListenedAlbums2023 = 
    dfWrapped2023
        .groupBy("album")
        .count()
        .rename("album").into("album2023")
        .rename("count").into("countYear2023")
        .sortByDesc("countYear2023")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")

In [9]:
val dfRanksAlbums2022 = 
    dfMostListenedAlbums2022
        .select { rank and album2022 }
        .rename("rank").into { "album2023_rank2022" }
        .rename("album2022").into { "album" }

In [10]:
// With the left join, albums from 2023 not present in 2022 will have
// the 2022 values filled in with null.
val dfMostListenedAlbums2023WithPreviousYearRank =
    dfMostListenedAlbums2023.leftJoin(dfRanksAlbums2022) { album2023 match right.album }

dfMostListenedAlbums2022
    .join(
        dfMostListenedAlbums2023WithPreviousYearRank
            .add("album2023_rank2023") { rank } 
    )

... showing only top 20 of 4355 rows DataFrame: rowsCount = 4355, columnsCount = 7

In [11]:
// Find the 10 most listened artists in 2022 and 2023, to compare.

val dfMostListenedArtists2022 = 
    dfWrapped2022
        .groupBy("artist")
        .count()
        .rename("artist").into("artist2022")
        .rename("count").into("countYear2022")
        .sortByDesc("countYear2022")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")

val dfMostListenedArtists2023 = 
    dfWrapped2023
        .groupBy("artist")
        .count()
        .rename("artist").into("artist2023")
        .rename("count").into("countYear2023")
        .sortByDesc("countYear2023")
        // Adds an increasing iterator per row, to be used as rank.
        .addId("rank")

In [12]:
val dfRanksArtists2022 = 
    dfMostListenedArtists2022
        .select { rank and artist2022 }
        .rename("rank").into { "artist2023_rank2022" }
        .rename("artist2022").into { "artist" }

In [13]:
// With the left join, albums from 2023 not present in 2022 will have
// the 2022 values filled in with null.
val dfMostListenedArtists2023WithPreviousYearRank =
    dfMostListenedArtists2023.leftJoin(dfRanksArtists2022) { artist2023 match right.artist }

dfMostListenedArtists2022
    .join(
        dfMostListenedArtists2023WithPreviousYearRank
            .add("artist2023_rank2023") { rank } 
    )

... showing only top 20 of 2844 rows DataFrame: rowsCount = 2844, columnsCount = 7

In [14]:
// Add columns for timestamps and date.
import java.sql.Timestamp

val dfWrappedWithDate2022 = 
    dfWrapped2022
        .add("timestamp") { Timestamp("uts"<Long>() * 1000 ) } 
        .add("date") { "timestamp"<Timestamp>().toString().substring(0..10) }

val dfWrappedWithDate2023 = 
    dfWrapped2023
        .add("timestamp") { Timestamp("uts"<Long>() * 1000 ) } 
        .add("date") { "timestamp"<Timestamp>().toString().substring(0..10) }

In [15]:
// Find the 10 days with the most songs listened in 2022 and 2023, to compare.

val dfDaysWithMostSongs2022 = 
    dfWrappedWithDate2022
        .groupBy("date")
        .count()
        .rename("date").into("day2022")
        .rename("count").into("countDay2022")
        .sortByDesc("countDay2022")
        .head(10)
        .addId("rank")

val dfDaysWithMostSongs2023 = 
    dfWrappedWithDate2023
        .groupBy("date")
        .count()
        .rename("date").into("day2023")
        .rename("count").into("countDay2023")
        .sortByDesc("countDay2023")
        .head(10)
        .addId("rank")

dfDaysWithMostSongs2022.join(dfDaysWithMostSongs2023)

DataFrame: rowsCount = 10, columnsCount = 5

In [16]:
// Find the 10 songs most listened the same day in 2022 and 2023, to compare.

val dfSongsMostRepeatedInADay2022 = 
    dfWrappedWithDate2022
        .groupBy{ "track" and "date" }
        .count()
        .rename("date").into("day2022")
        .rename("track").into("track2022")
        .rename("count").into("countDay2022")
        .sortByDesc("countDay2022")
        .head(10)
        .addId("rank")

val dfSongsMostRepeatedInADay2023 = 
    dfWrappedWithDate2023
        .groupBy{ "track" and "date" }
        .count()
        .rename("date").into("day2023")
        .rename("track").into("track2023")
        .rename("count").into("countDay2023")
        .sortByDesc("countDay2023")
        .head(10)
        .addId("rank")

dfSongsMostRepeatedInADay2022.join(dfSongsMostRepeatedInADay2023)

DataFrame: rowsCount = 10, columnsCount = 7

In [17]:
// Find the 10 albums most listened the same day in 2022 and 2023, to compare.

val dfAlbumsMostListenedInADay2022 = 
    dfWrappedWithDate2022
        .groupBy{ "album" and "date" }
        .count()
        .rename("date").into("day2022")
        .rename("album").into("album2022")
        .rename("count").into("countDay2022")
        .sortByDesc("countDay2022")
        .head(10)
        .addId("rank")

val dfAlbumsMostListenedInADay2023 = 
    dfWrappedWithDate2023
        .groupBy{ "album" and "date" }
        .count()
        .rename("date").into("day2023")
        .rename("album").into("album2023")
        .rename("count").into("countDay2023")
        .sortByDesc("countDay2023")
        .head(10)
        .addId("rank")

dfAlbumsMostListenedInADay2022.join(dfAlbumsMostListenedInADay2023)

DataFrame: rowsCount = 10, columnsCount = 7

In [18]:
// Find the 10 artists most listened the same day in 2022 and 2023, to compare.

val dfArtistsMostListenedInADay2022 = 
    dfWrappedWithDate2022
        .groupBy{ "artist" and "date" }
        .count()
        .rename("date").into("day2022")
        .rename("artist").into("artist2022")
        .rename("count").into("countDay2022")
        .sortByDesc("countDay2022")
        .head(10)
        .addId("rank")

val dfArtistsMostListenedInADay2023 = 
    dfWrappedWithDate2023
        .groupBy{ "artist" and "date" }
        .count()
        .rename("date").into("day2023")
        .rename("artist").into("artist2023")
        .rename("count").into("countDay2023")
        .sortByDesc("countDay2023")
        .head(10)
        .addId("rank")

dfArtistsMostListenedInADay2022.join(dfArtistsMostListenedInADay2023)

DataFrame: rowsCount = 10, columnsCount = 7

In [19]:
val countUniqueSongs2022 = (
    dfWrappedWithDate2022.groupBy("track").count()
).count().toFloat()

In [20]:
val countUniqueSongs2023 = (
    dfWrappedWithDate2023.groupBy("track").count()
).count().toFloat()

In [21]:
countUniqueSongs2022

7248.0

In [22]:
countUniqueSongs2023

6771.0

In [23]:
countUniqueSongs2023 / countUniqueSongs2022

0.9341887

In [24]:
val countUniqueArtists2022 = (
    dfWrappedWithDate2022.groupBy("artist").count()
).count().toFloat()

In [25]:
val countUniqueArtists2023 = (
    dfWrappedWithDate2023.groupBy("artist").count()
).count().toFloat()

In [26]:
countUniqueArtists2022

2844.0

In [27]:
countUniqueArtists2023

2953.0

In [28]:
countUniqueArtists2023 / countUniqueArtists2022

1.0383263

In [29]:
val countTotalSongs2022 = dfWrapped2022.count().toFloat()

In [30]:
val countTotalSongs2023 = dfWrapped2023.count().toFloat()

In [31]:
countTotalSongs2022

25351.0

In [32]:
countTotalSongs2023

17852.0

In [33]:
countTotalSongs2023 / countTotalSongs2022

0.7041931

In [34]:
%use lets-plot

In [35]:
val plotData2022 = 
    letsPlot(
        dfWrappedWithDate2022
            .sortBy("date")
            .toMap()
    ) { x = "date" } + ggsize(630, 500)
    
plotData2022 + geomHistogram(binWidth=0.5)

In [36]:
val plotData2023 = 
    letsPlot(
        dfWrappedWithDate2023
            .sortBy("date")
            .toMap()
    ) { x = "date" } + ggsize(630, 500)
    
plotData2023 + geomHistogram(binWidth=0.5)